In [21]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from Classes.Communicator import Communicator

diya_name = "06"
comm = Communicator(diya_name, save_data=True, save_frequency=10, choose_specific_directory=False, verbose=True)

In [22]:
import datetime

x_HP = 100
x_FAN = 0

continue_loop = True
initial_time = datetime.datetime.now()
duration = 60*10 # seconds

try:
    comm.start()
    comm.send_control_input(x_HP, x_FAN)

    while continue_loop:
        current_time = datetime.datetime.now()
        elapsed_time = current_time - initial_time
        if elapsed_time.total_seconds() > duration:
            continue_loop = False

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    comm.stop()

Communicator startedSubscribed to diya06/rx/

Message sent: x_HP = 100, x_FAN = 0
Received non-standard message
Received non-standard message
Received non-standard message
Received non-standard message
Received non-standard message
Received non-standard message
Received non-standard message
                             T1_C   T2_C   T3_C  x_HP  x_FAN  U_HP_V  I_HP_mA
2024-10-30 17:33:01.348443  44.65  44.85  63.69   100      0   394.0   1595.0
Received non-standard message
                             T1_C   T2_C   T3_C  x_HP  x_FAN  U_HP_V  I_HP_mA
2024-10-30 17:33:02.343821  44.65  44.85  63.69   100      0   394.0   1594.0
Received non-standard message
                             T1_C   T2_C  T3_C  x_HP  x_FAN  U_HP_V  I_HP_mA
2024-10-30 17:33:03.302098  44.66  44.85  63.7   100      0   394.0   1595.0
Received non-standard message
                             T1_C   T2_C   T3_C  x_HP  x_FAN  U_HP_V  I_HP_mA
2024-10-30 17:33:04.295202  44.67  44.85  63.71   100      0   402.0   159

In [ ]:
import paho.mqtt.client as mqtt

# Define the MQTT broker details
broker = "mqtt.119.ovh"
port = 1883
topic = "diya06"
read_topic = topic + "/rx/"
transmit_topic = topic + "/tx/"
keep_alive = 60

last_message = None

# Callback function when a message is received
def on_message(client, userdata, message):
    last_message = message.payload.decode()
    # print(f"Message received: {last_message} on topic {message.topic}")

    parts = last_message.split()
    if "TMP&U" not in last_message or "Peltier:" not in last_message or "Fan:" not in last_message:
        print("Received incomplete message")
        return
    
    # Parse each component based on its position in the message
    T_1 = float(parts[2])
    T_2 = float(parts[3])
    T_3 = float(parts[4])

    # Parse the 'Peltier' and 'Fan' values by removing their labels
    peltier_value = int(parts[6])
    fan_value = int(parts[8])

    # Print out parsed values
    print("Measurements:", T_1, T_2, T_3)
    print("Peltier Value:", peltier_value)
    print("Fan Value:", fan_value)
    print("-------------------------------------------")

def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
    sign_HP = "+" if x_HP >= 0 else "-"

    # Bound control values
    x_HP = max(min(x_HP, 100), -100)
    x_FAN = max(min(x_FAN, 100), 0)

    # Format control values
    x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
    x_FAN_formatted = f"{int(x_FAN):03d}"

    message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

    client.publish(transmit_topic, message_str)
    print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")


# Create an MQTT client instance
# cself.client = mqtt.Client(api_version=mqtt.CallbackAPIVersion.VERSION2) ##################################3
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(read_topic)

In [ ]:
x_HP = 0
x_FAN = 0

send_control_input(client, x_HP, x_FAN)

client.loop_start()

In [ ]:
client.loop_stop()

In [ ]:
client.disconnect()